# ノック31

In [1]:
import pandas as pd
from sklearn.cluster import KMeans
from scipy import stats
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

データを読み込む

In [2]:
use_log = pd.read_csv("use_log.csv")
use_log.head()

,log_id,customer_id,usedate
0,L00000049012330,AS009373,2018-04-01
1,L00000049012331,AS015315,2018-04-01
2,L00000049012332,AS040841,2018-04-01
3,L00000049012333,AS046594,2018-04-01
4,L00000049012334,AS073285,2018-04-01


In [3]:
customer = pd.read_csv("customer_join.csv")
customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,ナイト,6000,通常,3.916667,4.0,6,1,1,2019-04-30,47


欠損値の確認

In [4]:
use_log.isnull().sum()

log_id         0
customer_id    0
usedate        0
dtype: int64

In [5]:
customer.isnull().sum()

customer_id             0
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
dtype: int64

# ノック32(ここから自身の解答)

クラスタリングを行う

まずは、mean,median,max,min,membership_periodを分ける

In [6]:
customer_clustering = customer[["mean","median","max","min","membership_period"]]
customer_clustering.head()

,mean,median,max,min,membership_period
0,4.833333,5.0,8,2,47
1,5.083333,5.0,7,3,47
2,4.583333,5.0,6,3,47
3,4.833333,4.5,7,2,47
4,3.916667,4.0,6,1,47


K平均法を使用する前に、membership_periodを標準化する

In [7]:
customer_clustering["membership_period"].iloc[:].values.reshape(-1,1)

array([[47],
       [47],
       [47],
       ...,
       [ 1],
       [ 1],
       [ 1]], dtype=int64)

In [8]:
customer_clustering.head()

,mean,median,max,min,membership_period
0,4.833333,5.0,8,2,47
1,5.083333,5.0,7,3,47
2,4.583333,5.0,6,3,47
3,4.833333,4.5,7,2,47
4,3.916667,4.0,6,1,47


In [9]:
# StandardScalerを用いて標準化

scaler = StandardScaler()
scaler.fit(customer_clustering["membership_period"].values.reshape(-1,1))
customer_clustering["membership_period"] = scaler.transform(customer_clustering["membership_period"].values.reshape(-1,1))

C:\Users\Takeshi\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [10]:
# 標準化出来ているかを確認

customer_clustering["membership_period"]

0       2.034025
1       2.034025
2       2.034025
3       2.034025
4       2.034025
          ...   
4187   -1.288115
4188   -1.288115
4189   -1.288115
4190   -1.288115
4191   -1.288115
Name: membership_period, Length: 4192, dtype: float64

K平均法にて4つのグループにクラスター分け

In [11]:
customer_clustering.head()

,mean,median,max,min,membership_period
0,4.833333,5.0,8,2,2.034025
1,5.083333,5.0,7,3,2.034025
2,4.583333,5.0,6,3,2.034025
3,4.833333,4.5,7,2,2.034025
4,3.916667,4.0,6,1,2.034025


In [15]:
pred_cluster =  KMeans(n_clusters=4).fit_predict(customer_clustering)
customer_clustering["cluster"] = pred_cluster
customer_clustering

C:\Users\Takeshi\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,mean,median,max,min,membership_period,cluster
0,4.833333,5.0,8,2,2.034025,0
1,5.083333,5.0,7,3,2.034025,0
2,4.583333,5.0,6,3,2.034025,0
3,4.833333,4.5,7,2,2.034025,0
4,3.916667,4.0,6,1,2.034025,0
...,...,...,...,...,...,...
4187,8.000000,8.0,8,8,-1.288115,1
4188,10.000000,10.0,10,10,-1.288115,1
4189,8.000000,8.0,8,8,-1.288115,1
4190,11.000000,11.0,11,11,-1.288115,1


In [13]:
# 0～3の個数内訳の確認

customer_clustering["cluster"].value_counts()

3    1946
0    1044
1     707
2     495
Name: cluster, dtype: int64